# Exploration of the auto_mpg data set

This notebook presents an exploration of the auto_mpg dataset, using python and pandas. This is the same data set you were asked to investigate using your preferred tools.

Students are recommended to run the notebook one cell at a time using the "play" button above.

Note that the notebook assumes that the data is placed in a `data/` subfolder of the folder where the notebook has been placed. This convention is common in machine learning.

_For convenience, here is the spec from the previous exercise:_

For each attribute (column) find the following information.
  * The attribute type, e.g. nominal, ordinal, numeric.
  * Percentage of missing values in the data.
  * Statistical numerical measures - centre (mean, median, mode), spread (min, max, range, standard deviation), symmetry (skewness).
  * Statistical graphical representations &mdash; bar plots, histograms, ... even pie charts, although they can be difficult to interpret, and [better alternatives exist](https://www.data-to-viz.com/caveat/pie.html).
  * Are there any rows that have a value for the attribute that no other record has (i.e. unique values)?
  * Are there any outliers?
  * Which attributes seem to be linked?

In [ ]:
import pandas as pd
auto_mpg = pd.read_csv('data/auto-mpg.csv')
auto_mpg.head()

## Display the data type of each column in the dataframe.

`.dtypes()` returns a pandas _Series.

In [ ]:
auto_mpg.dtypes

`float64` and `int64` are numeric columns. An object column means that python was unable to decide what datatype the column contains. It is likely that it is a mixed type, so we can look at the individual values.

In [ ]:
auto_mpg["car name"].value_counts()

Note that some cars appears multiple times, so it is interesting to see why that might happen.

In [ ]:
auto_mpg.loc[auto_mpg['car name'] == 'ford pinto']

Note that the 'ford pinto' cars differ by model year. So in this case, it suggests the `car name` is a simple description of the car, but is not enough on its own to be used to identify individual rows in the dataframe. In database terms, it is not enough to act as a primary key for the dataframe.

Also notice that the other `object` type column (i.e., `horsepower`) contains numeric values and '?', which is often used as a placeholder for missing data. We can take a look at the column to assess what it contains.

In [ ]:
auto_mpg["horsepower"].values

As can be seen, `horsepower` generally contains numeric data but this data is stored as strings, because some `horsepower` values are represented using `'?'`. So if we wish to work with this data we should

1. remove the `'?'` placeholders - this is missing data and should be recognised as such
2. convert the remaining values from string to numeric, so that they match the intended use of this column.

In later weeks, we will cover how to perform these "data fixes" using pandas and python more generally.

## Looking for missing values

The usual way to look for missing values is to look for null values using the `.isnull()` function. For floating point data we can also use the `.isna` function, which checks whether a floating point value is NaN (Not a Number).

We can apply `.isnull()` to an entire dataframe, or to a row, or a column or indeed a single value. The function returns a dataframe, a row series, a column series of `bool` values, as appropriate. In python, a `bool` value of `False` is treated as `0` and a `bool` value of `True` is treats as 1. We can count the `.isnull() == True` values by summing them using `.sum()`.

Firstly, we look across the entire dataframe.

In [ ]:
auto_mpg.isnull().sum().sum()

Here is how we can apply `.isnull()` and `sum()` to count the number of null values in the `auto_mpg["horsepower"]` column. Note that there appear to be none, owing to their replacement with `'?'`.

In [ ]:
auto_mpg["horsepower"].isnull().sum()

Perhaps a better check in this case is to count the number of `'?'` characters in the column.

In [ ]:
pd.Series(auto_mpg["horsepower"]=='?').sum()

So there are 6 rows with missing horsepower values.... The number of rows can be obtained from the `.shape` tuple, as follows, where it is clear that there are 398 rows and 9 columns.

In [ ]:
auto_mpg.shape

So the percentage of rows missing a value can be calculated using

In [ ]:
numMissingRows = pd.Series(auto_mpg["horsepower"]=='?').sum()
numRows = auto_mpg.shape[0]
percentMissingRows = round(100*numMissingRows/numRows,1)
percentMissingRows

## Statistical numerical measures

For the numeric-valued columns, we can compute a number of statistics (measures). Just using the `weight` column as an example, here is how you could get several measures at once:

In [ ]:
auto_mpg["weight"].describe()

Alternatively, you could calculate the mean of several columns using the following:

In [ ]:
auto_mpg[["displacement","weight"]].mean()

Note that `["displacement","weight"]` is a (python) list containing the two column names.

## Statistical plots

Plots greatly help data exploration. With experience, it is possible to observe many useful aspects, including:

1. distribution of a single variable
3. relationship of one variable against another

Again we will use `displacement` (engine size) and `weight` as the variables we wish to consider.

Python does not have built-in plotting capability, so we need to import some "standard" plotting libraries use in python-based machine learning. The first of these is `matplotlib` which is comprehensive but can take effort to configure the output. The second is `seaborn` which provides a higher-level abstraction, but requires `matplotlib` to support its operations. `seaborn` has many styling options, just showing one here....

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

The first 2 plots show the distribution of `weight` and `displacement`, each as a histogram. The plots are placed side-by-side to make them easier to compare.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('weight and displacement histograms')
v='weight'
sns.histplot(ax=ax1, x=auto_mpg[v])
ax1.set_xlabel(v)
v='displacement'
sns.histplot(ax=ax2, x=auto_mpg[v])
ax1.set_xlabel(v)

The origin is coded as 1:USA, 2:Europe and 3:Japan. To make the plots easier to understand, we will add a `country` column, as follows:

In [ ]:
auto_mpg['country'] = auto_mpg['origin'].replace([1,2,3],['USA','Europe','Japan'])
auto_mpg['country'].value_counts()

Note that is is good practice to check that an operation like this was successful. It is also noticeable that most of the cars in the dataset were manufactured in the USA.

Now we can create a boxplot of `displacement` and of `weight`, and overlay it with the overall median of that variable.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('weight and displacement boxplots')
v = 'weight'
sns.boxplot(ax=ax1, x='country', y=v, data=auto_mpg)
ax1.set_xlabel(v)
ax1.axhline(auto_mpg[v].median(), color='r', linestyle='dotted')
v = 'displacement'
sns.boxplot(ax=ax2, x='country', y=v, data=auto_mpg)
ax2.set_xlabel(v)
ax2.axhline(auto_mpg[v].median(), color='r', linestyle='dotted')

Clearly 1970s cars from the USA had considerably larger engines and were much heavier than their European or Japanese equivalents!

Lastly we will look at how 'displacement` and `weight` are related. Again, we will highlight the country of origin, by using a different colour for each country in the scatterplot below.

We will also show a trend line for each subset, noting that the trend depends on the country.

Lastly, we can also save the plot in a file as a portable network graphic (PNG) file. Note that `savefig()` uses the file extension to select the type of output graphic file.

In [ ]:
plot = sns.lmplot(x='displacement', y='weight', data=auto_mpg, hue='country')
plot.savefig("weightVSdisplacement.png")

Assuming there are not too many numerical columns, we can generate a pairwise scatterplot, to see at a glance the relationships in the data.

In [ ]:
sns.set()
sns.pairplot(auto_mpg, hue ='country')
plt.show()

Studying this pairsplot yields many insights into the data. We can choose more specific plots to learn more and check our understanding.

## Are there any rows that have a value for the attribute that no other record has (i.e. unique values)?

We have already seen how to get a count of specific values in each column. We are looking for columns that have a value that appears only once.

We can loop over the columns in the dataframe and use `.value_counts()` on each volumn.

In [ ]:
for col in auto_mpg.columns:
  print(auto_mpg[col].value_counts())

As can be seen, columns like `mpg`, `displacement`, `horsepower`, `weight` and `acceleration` include unique values, but that might be expected. It would be more significant in columns where the number of values is limited.

## Are there any outliers?

Yes. There are many techniques for finding outliers, but one of the easiest ways to find outliers from a single variable is to use boxplots.

As can be seen in the boxplots earlier in this notebooks, `displacement` has some upper outliers in the case of European cars.

## Which attributes seem to be linked?

The pairsplot earlier in this notebook identifies some linear relationships. e.g., between `mpg` vs `weight`, `displacement` and `acceleration`.

There are other pairs that have no or at best a very weak linear relationship, because the scatterplots look more like a cloud. Examples of these include `displacement` versus `model year`.

## Overall

This notebook is an example of how notebook can be used to answer questions about a nontrivial dataset.

Some takeaways:

- The notebook is structured in cells, where an analysis is motivated and described in a markdown cell, the code is then run and produces an output, and this is followed by another markdown cell analysing what was found.
- `pandas` and `python` together can be used to manipulate the data, and to prepare simple reports
- `matplotlib` and `seaborn` can be used to produce plots